<a href="https://colab.research.google.com/github/rajanchoksi79/pytorch_practice/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# checking version
torch.__version__


'2.6.0+cu124'